In [1]:
import os
import requests
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.tools import Tool
from langchain.chat_models import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
import warnings
warnings.filterwarnings("ignore")
import google.generativeai as genai
from langchain_community.tools import DuckDuckGoSearchRun,WikipediaQueryRun
from dotenv import load_dotenv


load_dotenv()


GEMINI_API_KEY = os.getenv("GOOGLE_API_KEY")
MAPS_API_KEY = os.getenv("google_maps_api_key")



c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\importlib\__init__.py:126: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  return _bootstrap._gcd_import(name[level:], package, level)
c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
  warnings.warn(message, UserWarning)


In [2]:
api_wrapper = WikipediaAPIWrapper(top_k_results=3)

In [4]:
#Tourism
def get_lat_lng(location_name):
    geocode_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={location_name}&key={MAPS_API_KEY}"
    response = requests.get(geocode_url)
    if response.status_code == 200:
        geo_data = response.json()
        if geo_data['results']:
            location = geo_data['results'][0]['geometry']['location']
            lat_lng = f"{location['lat']},{location['lng']}"
            return lat_lng
        else:
            return "Location not found."
    else:
        return "Error fetching location data."

# Function to fetch nearby tourist places using Google Places API
def get_tourist_places(location_name):
    lat_lng = get_lat_lng(location_name)
    if "Error" in lat_lng or "Location not found" in lat_lng:
        return lat_lng

    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": lat_lng,
        "radius": 30000,        # Search radius in meters (5km)
        "type": "tourist_attraction",
        "key": MAPS_API_KEY
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        places = response.json().get('results', [])
        if places:
            place_names = [place['name'] for place in places[:5]]
            return place_names
        else:
            return "No tourist attractions found nearby."
    else:
        return f"Could not retrieve tourist places for {location_name}. Please try another location."

# Function to fetch brief history or summary from Wikipedia for a tourist place
def get_place_history(place_name):
    wikipedia_url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{place_name.replace(' ', '_')}"
    response = requests.get(wikipedia_url)
    if response.status_code == 200:
        data = response.json()
        return data.get("extract", "No history available.")
    else:
        return f"Could not retrieve history for {place_name}."

# Tool to get nearby tourist places with history
def get_places_with_history(location_name):
    places = get_tourist_places(location_name)
    if isinstance(places, str):  # Handle error messages from get_tourist_places
        return places

    formatted_places_with_history = []
    for place in places:
        history = get_place_history(place)
        formatted_places_with_history.append(f"{place}:\n{history}")

    # Join all place information in a formatted output
    return "\n\n".join(formatted_places_with_history)

# Define the tool for getting tourist places with history
tourist_places_with_history_tool = Tool(
    name="tourist_places_with_history",
    func=get_places_with_history,
    description="Provides a list of nearby tourist places along with a brief history for each place."
)

# Initialize the language model (OpenAI GPT-3.5 or GPT-4)


# Set up memory for personalized conversations
memory = ConversationBufferMemory()

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
)
# Create the agent using LangChain
agent = initialize_agent(
    tools=[tourist_places_with_history_tool],  # Add tool for places and history
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

# Ask the user for a location
#user_query = input("Enter the location for which you'd like recommendations of tourist places with a brief history: ")
user_query = input("Enter a city to receive recommendations for must-visit tourist places and a brief history for each recommended spot: ")

# Fetch and print the tourist places with history for the selected location
response_each_places_with_history = agent.run(user_query)
print(f"Tourist places in {user_query}:\n{response_each_places_with_history}")




> Entering new AgentExecutor chain...
Action: tourist_places_with_history
Action Input: Dubai
Observation: Wild Wadi Waterpark Jumeirah:
Could not retrieve history for Wild Wadi Waterpark Jumeirah.

Ski Dubai:
Ski Dubai is an indoor ski resort with 22,500 square meters of indoor ski area. The park maintains a temperature of −1 to 2 °C throughout the year. It is a part of the Mall of the Emirates, one of the largest shopping malls in the world, located in Dubai, United Arab Emirates. It was developed by Majid Al Futtaim Group, which also operates the Mall of the Emirates.

Quad Biking Dubai and Dune Buggy Ride:
Could not retrieve history for Quad Biking Dubai and Dune Buggy Ride.

Mushrif Park:
Mushrif Park is a family-oriented park in Dubai, United Arab Emirates. It is located in the eastern part of the city, about 16 km from the traditional center of Dubai. The park was created in 1982 by Dubai Municipality and was widely expanded and refurbished in 1989. Facilities include a swimmi